In [ ]:
# 测试读取数据

In [11]:
import geopandas as gpd


shp_path = "shpfile/xian.shp"
shp_gpd = gpd.read_file(shp_path, encoding='utf8')

In [12]:
shp_gpd

,FEATUREGUI,FCODE,NAME,FSCALE,USOURCE,UPDATETIME,UPDATESTAT,SHAPE_Leng,SHAPE_Area,COUNTY,SHI,TYPE,CITYCODE,geometry
0,be282a648f2f42fbac7c77a6e01ce138,6501000500,路桥区,11,20190402基础库全量包,20190402,A,1.774333,0.029110,路桥区,台州市,3,331004,"MULTIPOLYGON (((121.63216 28.45817, 121.63206 ..."
1,982e7f2b0ac040d788a122ec4c1655cf,6501000500,黄岩区,11,20190402基础库全量包,20190402,A,2.437395,0.091147,黄岩区,台州市,3,331003,"POLYGON ((121.19271 28.72962, 121.19343 28.729..."
2,8e13cd9345774f0799b0250db762c879,6501000500,仙居县,11,20190402基础库全量包,20190402,A,3.253885,0.184719,仙居县,台州市,3,331024,"POLYGON ((120.77655 28.99437, 120.77785 28.993..."
3,3baea72264c7458cb77443e36541c75c,6501000500,天台县,11,20190402基础库全量包,20190402,A,2.696434,0.132769,天台县,台州市,3,331023,"POLYGON ((121.25211 29.11772, 121.25040 29.116..."
4,ea95fb32bfc24221adfd301b0f76bc7c,6501000500,三门县,11,2020年6月基础库政区数据,20200611,A,5.224158,0.094042,三门县,台州市,3,331022,"MULTIPOLYGON (((121.75236 28.87361, 121.75250 ..."
5,88d9c8c792364811ace57eddfcadd403,6501000500,临海市,11,2020年6月基础库政区数据,20200611,A,5.321144,0.202621,临海市,台州市,3,331082,"MULTIPOLYGON (((121.78788 28.64356, 121.78800 ..."
6,d44d3911493c4c51a1ff44cbacef629c,6501000500,椒江区,11,2020年6月基础库政区数据,20200611,A,2.458704,0.030184,椒江区,台州市,3,331002,"MULTIPOLYGON (((121.90887 28.39064, 121.90949 ..."
7,107010539f574c938bfe15fbba1ceafa,6501000500,温岭市,11,2020年6月基础库政区数据,20200611,A,4.444571,0.089637,温岭市,台州市,3,331081,"MULTIPOLYGON (((121.55381 28.20993, 121.55371 ..."
8,204c3ad21f8746bea8271e59570bb35b,6501000500,玉环市,11,2020年6月基础库政区数据,20200611,A,3.428621,0.041509,玉环市,台州市,3,331083,"MULTIPOLYGON (((121.14731 28.02226, 121.14715 ..."


In [7]:
# coding=UTF-8
import gevent
from math import isnan
import numpy as np
#import modin.pandas as pd 
import pandas as pd
import time
import netCDF4 
# import h5netcdf.legacyapi as netCDF4
import h5py

from mpl_toolkits.basemap import Basemap
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import shapefile
import matplotlib as mpl
import xarray as xr
from matplotlib.font_manager import FontProperties
import netCDF4 as nc
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib
import geopandas as gpd
from ncmaps import Cmaps
from io import BytesIO
import base64
import json
import math
from scipy.interpolate import griddata
from math import ceil, floor
from rasterio import features
from affine import Affine
import os
from datetime import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from matplotlib.font_manager import FontProperties  # 步骤一

os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'

basicfile = '/home/liyuan3970/Data/My_Git/web_met/'

class canvas_plot:
    def __init__(self, plot_self_data,plot_title,plot_bar):
        self.plot_self_data = plot_self_data
        self.plot_title = plot_title
        self.plot_bar = plot_bar
        self.data = self.read_data()
    # 外部函数
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    def basemask(self,cs, ax, map, shpfile):
        sf = shapefile.Reader(shpfile)
        vertices = []
        codes = []
        for shape_rec in sf.shapeRecords():
            if shape_rec.record[0] >= 0:  
                pts = shape_rec.shape.points
                prt = list(shape_rec.shape.parts) + [len(pts)]
                for i in range(len(prt) - 1):
                    for j in range(prt[i], prt[i+1]):
                        vertices.append(map(pts[j][0], pts[j][1]))
                    codes += [Path.MOVETO]
                    codes += [Path.LINETO] * (prt[i+1] - prt[i] -2)
                    codes += [Path.CLOSEPOLY]
                clip = Path(vertices, codes)
                clip = PathPatch(clip, transform = ax.transData)    
        for contour in cs.collections:
            contour.set_clip_path(clip)    
    def makedegreelabel(self,degreelist):
        labels=[str(x)+u'°E' for x in degreelist]
        return labels
    def village_data(self):
        data_xr = self.data
        shp_path = "shpfile/"
        shp_data = gpd.read_file("shpfile/xiangzhen.shp", encoding='utf8')
        village_list = shp_data['NAME'].values
        shp_da = self.add_shape_coord_from_data_array(data_xr, shp_path+"xiangzhen.shp", "country")
        data_dir = {}
        for i in range(len(village_list)):
            awash_da = shp_da.where(shp_da.country==i, other=np.nan)
            name = village_list[i]
            if np.isnan(awash_da.mean().values.tolist()):
                data_dir[name]  = 0.0
            else:
                data_dir[name]  = awash_da.mean().values.tolist()
        key = []
        value = []
        for item in data_dir.items():
            key.append(item[0])
            value.append(item[1])     
        return key,value
    def shp_average(self):
        data_xr = self.data
        shp_path = "shpfile/"
        shp_data = gpd.read_file("shpfile/taizhou.shp", encoding='utf8')
        county_list = shp_data['NAME_3'].values
        shp_da = self.add_shape_coord_from_data_array(data_xr, shp_path+"taizhou.shp", "country")
        data_list = []
        for i in range(len(county_list)):
            awash_da = shp_da.where(shp_da.country==i, other=np.nan)
            data_list.append(awash_da.mean().values.tolist())
        return data_list
        
    def read_data(self):
        data = self.plot_self_data
        a = []
        b = []
        z = []
        for i in range(len(data['station'])):
            if data['station'][i][2]!=-9999.0:
                a.append(data['station'][i][0])
                b.append(data['station'][i][1])
                z.append(data['station'][i][2])
        lat = np.array(b)
        lon = np.array(a)
        Zi = np.array(z)
        data_max = max(Zi)
        data_min = min(Zi)
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.05)
        y = np.arange(27.8,29.5,0.05)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        data_xr = xr.DataArray(Z_linear, coords=[ y,x], dims=["lat", "lon"])
        return data_xr   

    

plot_self_data = {
        "station_list": ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'],
        "station": [
            [120.5, 28.5, 99],
            [120.92, 28.78, 58.3],
            [120.47, 28.55, 45],
            [120.91, 29.11, 35],
            [121.46, 28.94, 25],
            [121.05, 28.77, 15],
            [121.41, 28.64, 25],
            [120.21, 28.66, 35],
            [121.41, 28.47, 45],
            [121.40, 28.34, 55],
            [121.15, 27.94, 65],
            [121.21, 28.41, 75],
            [120.54, 28.70, 85],
            [121.20, 28.68, 15],
            [120.66, 28.89, 25],
            [121.04, 29.00, 23],
            [121.25, 28.90, 23],
            [121.21, 28.20, 65],
            [121.15, 28.25, 68],
            [120.95, 28.60, 68],
            [121.00, 28.60, 99]

        ]
    }
    
plot_title,plot_bar  = "降水预报测试画图" ,[0,1,2,3,4,5,6,7]  
worker = canvas_plot(plot_self_data,plot_title,plot_bar)
key,value = worker.village_data()
# data = worker.shp_average()

In [8]:
value

[25.0,
 25.0,
 45.0,
 25.0,
 25.0,
 25.0,
 65.0,
 0.0,
 0.0,
 50.0,
 85.0,
 40.0,
 31.0,
 35.0,
 42.15,
 23.0,
 23.0,
 23.0,
 24.333333333333332,
 21.0,
 23.0,
 21.857142857142858,
 23.0,
 15.0,
 25.0,
 25.0,
 85.0,
 85.0,
 85.0,
 65.0,
 72.26,
 15.0,
 41.65,
 15.0,
 99.0,
 55.0,
 55.0,
 55.0,
 0.0,
 45.0,
 15.0,
 15.0,
 57.0,
 15.0,
 25.0,
 25.0,
 45.0,
 0.0,
 15.0,
 45.0,
 45.0,
 50.0,
 0.0,
 65.0,
 0.0,
 58.3,
 58.3,
 35.0,
 40.0,
 25.0,
 31.666666666666668,
 25.0,
 28.333333333333332,
 51.083333333333336,
 63.84285714285714,
 68.0,
 80.75,
 71.85714285714286,
 70.55555555555556,
 39.0,
 45.0,
 68.0,
 68.0,
 99.0,
 99.0,
 15.0,
 75.0,
 60.0,
 0.0,
 17.0,
 25.0,
 15.0,
 15.0,
 15.0,
 25.825,
 25.0,
 25.0,
 0.0,
 45.0,
 45.0,
 33.5,
 23.0,
 25.0,
 23.0,
 23.8,
 25.0,
 23.4,
 24.5,
 23.0,
 23.0,
 26.0,
 35.0,
 35.0,
 0.0,
 99.0,
 35.0,
 35.0,
 25.0,
 15.0,
 0.0,
 65.0,
 66.5,
 75.0,
 55.0,
 55.0,
 55.0,
 25.0,
 65.0,
 25.0,
 65.0,
 45.0,
 25.0,
 0.0,
 68.0,
 65.0,
 65.0,
 0.0,
 55.0,
 

In [58]:
data

[('古城街道', 15.0),
 ('新前街道', 15.0),
 ('江南街道', 15.0),
 ('高桥街道', 15.0),
 ('澄江街道', 15.0),
 ('西城街道', 15.0),
 ('南城街道', 15.0),
 ('永丰镇', 21.0),
 ('邵家渡街道', 21.857142857142858),
 ('河头镇', 23.0),
 ('坦头镇', 23.0),
 ('汇溪镇', 23.0),
 ('大田街道', 23.0),
 ('大洋街道', 23.0),
 ('海游街道', 24.333333333333332),
 ('安洲街道', 25.0),
 ('桃渚镇', 25.0),
 ('杜桥镇', 25.0),
 ('海门街道', 25.0),
 ('三甲街道', 25.0),
 ('福应街道', 25.0),
 ('南峰街道', 25.0),
 ('赤城街道', 31.0),
 ('始丰街道', 35.0),
 ('官路镇', 40.0),
 ('大战乡', 41.65),
 ('蓬街镇', 45.0),
 ('清港镇', 65.0),
 ('芦浦镇', nan),
 ('石桥头镇', nan),
 ('葭沚街道', 25.0),
 ('章安街道', 25.0),
 ('海润街道', 25.0),
 ('广度乡', 25.0),
 ('龙溪乡', 28.333333333333332),
 ('街头镇', 31.666666666666668),
 ('白鹤镇', 35.0),
 ('安岭乡', 40.0),
 ('白水洋镇', 42.15),
 ('螺洋街道', 45.0),
 ('路南街道', 45.0),
 ('路桥街道', 45.0),
 ('泽国镇', 45.0),
 ('箬横镇', 50.0),
 ('新河镇', 50.0),
 ('太平街道', 55.0),
 ('城西街道', 55.0),
 ('城东街道', 55.0),
 ('头陀镇', 57.0),
 ('步路乡', 65.0),
 ('屿头乡', 72.26),
 ('埠头镇', 85.0),
 ('白塔镇', 85.0),
 ('皤滩乡', 85.0),
 ('上垟乡', 99.0),
 ('横峰街道', nan),
 ('路北街道', nan),
 

In [51]:
data
# d_result=sorted(data.items(),key=lambda x:x[1],reverse=False)

{'安洲街道': 25.0,
 '桃渚镇': 25.0,
 '蓬街镇': 45.0,
 '杜桥镇': 25.0,
 '海门街道': 25.0,
 '三甲街道': 25.0,
 '清港镇': 65.0,
 '芦浦镇': nan,
 '石桥头镇': nan,
 '箬横镇': 50.0,
 '埠头镇': 85.0,
 '官路镇': 40.0,
 '赤城街道': 31.0,
 '始丰街道': 35.0,
 '白水洋镇': 42.15,
 '河头镇': 23.0,
 '坦头镇': 23.0,
 '汇溪镇': 23.0,
 '海游街道': 24.333333333333332,
 '永丰镇': 21.0,
 '大田街道': 23.0,
 '邵家渡街道': 21.857142857142858,
 '大洋街道': 23.0,
 '古城街道': 15.0,
 '福应街道': 25.0,
 '南峰街道': 25.0,
 '白塔镇': 85.0,
 '皤滩乡': 85.0,
 '田市镇': 85.0,
 '步路乡': 65.0,
 '屿头乡': 72.26,
 '新前街道': 15.0,
 '大战乡': 41.65,
 '江南街道': 15.0,
 '上垟乡': 99.0,
 '太平街道': 55.0,
 '城西街道': 55.0,
 '城东街道': 55.0,
 '横峰街道': nan,
 '螺洋街道': 45.0,
 '高桥街道': 15.0,
 '澄江街道': 15.0,
 '头陀镇': 57.0,
 '西城街道': 15.0,
 '葭沚街道': 25.0,
 '章安街道': 25.0,
 '路南街道': 45.0,
 '路北街道': nan,
 '南城街道': 15.0,
 '路桥街道': 45.0,
 '泽国镇': 45.0,
 '新河镇': 50.0,
 '横街镇': nan,
 '楚门镇': 65.0,
 '龙溪镇': nan,
 '下各镇': 58.3,
 '双庙乡': 58.3,
 '白鹤镇': 35.0,
 '安岭乡': 40.0,
 '海润街道': 25.0,
 '街头镇': 31.666666666666668,
 '广度乡': 25.0,
 '龙溪乡': 28.333333333333332,
 '括苍镇': 51.083333333333336,
 '朱溪镇

In [50]:
d_result

[('古城街道', 15.0),
 ('新前街道', 15.0),
 ('江南街道', 15.0),
 ('高桥街道', 15.0),
 ('澄江街道', 15.0),
 ('西城街道', 15.0),
 ('南城街道', 15.0),
 ('永丰镇', 21.0),
 ('邵家渡街道', 21.857142857142858),
 ('河头镇', 23.0),
 ('坦头镇', 23.0),
 ('汇溪镇', 23.0),
 ('大田街道', 23.0),
 ('大洋街道', 23.0),
 ('海游街道', 24.333333333333332),
 ('安洲街道', 25.0),
 ('桃渚镇', 25.0),
 ('杜桥镇', 25.0),
 ('海门街道', 25.0),
 ('三甲街道', 25.0),
 ('福应街道', 25.0),
 ('南峰街道', 25.0),
 ('赤城街道', 31.0),
 ('始丰街道', 35.0),
 ('官路镇', 40.0),
 ('大战乡', 41.65),
 ('蓬街镇', 45.0),
 ('清港镇', 65.0),
 ('芦浦镇', nan),
 ('石桥头镇', nan),
 ('葭沚街道', 25.0),
 ('章安街道', 25.0),
 ('海润街道', 25.0),
 ('广度乡', 25.0),
 ('龙溪乡', 28.333333333333332),
 ('街头镇', 31.666666666666668),
 ('白鹤镇', 35.0),
 ('安岭乡', 40.0),
 ('白水洋镇', 42.15),
 ('螺洋街道', 45.0),
 ('路南街道', 45.0),
 ('路桥街道', 45.0),
 ('泽国镇', 45.0),
 ('箬横镇', 50.0),
 ('新河镇', 50.0),
 ('太平街道', 55.0),
 ('城西街道', 55.0),
 ('城东街道', 55.0),
 ('头陀镇', 57.0),
 ('步路乡', 65.0),
 ('屿头乡', 72.26),
 ('埠头镇', 85.0),
 ('白塔镇', 85.0),
 ('皤滩乡', 85.0),
 ('上垟乡', 99.0),
 ('横峰街道', nan),
 ('路北街道', nan),
 

In [44]:
d_result[1][1].astype(float)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [46]:
d_result[1][1].tolist()

15.0

# ec数据的测试

In [ ]:
# 